In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
import time
#from pytimekr import pytimekr

In [ ]:
!pip install pytimekr

  Preparing metadata (setup.py) ... done
  Created wheel for pytimekr: filename=pytimekr-0.1.0-py3-none-any.whl size=7922 sha256=d529aa07060896a64bc0107a8321ca8ab6fabda72ad6442be671cf5e31f7fcb7
  Stored in directory: /root/.cache/pip/wheels/b3/a3/ba/e73aaae1dec550b8667510c379b019295bba855275258745de
Successfully built pytimekr


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 데이터 확인

In [6]:
# vod
vod_df = pd.read_csv('/content/drive/MyDrive/hv_project/5_Data/데이터스쿨3차_2308월/데이터스쿨_3차_VOD_2308.csv', encoding='euc-kr', sep='\t')
content_df = pd.read_csv('/content/drive/MyDrive/hv_project/5_Data/데이터스쿨3차_2308월/데이터스쿨_3차_콘텐츠_2308.csv', encoding='euc-kr', sep='\t')

vod_df.head()

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt
0,65941000,(HD)그것이알고싶다 1361회(23/07/22),TV 시사/교양,기타,4800,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,1:20,20230812163507
1,66873000,(HD)그것이알고싶다 1361회(23/07/22),TV 시사/교양,기타,4800,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,1:20,20230816205227
2,66873000,(HD)그것이알고싶다 1361회(23/07/22),TV 시사/교양,기타,718,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,1:20,20230829194727
3,61689000,(HD)그것이알고싶다 1361회(23/07/22),TV 시사/교양,기타,4800,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,1:20,20230813130609
4,61619000,꼬리에꼬리를무는그날이야기 37회(22/07/21),TV 시사/교양,기타,4183,"살인범의 미토콘드리아 - 2006 냉동고 살인사건. 2006년 7월 23일, 서울 ...","장도연,장현성,장성규",1:13,20230804092737


In [ ]:
# channel

channel_df = pd.read_csv('/content/drive/MyDrive/hv_project/Data/channel.csv',
                         encoding="cp949", names=['subsr','channel','strt_dt','use_tms','asset_nm'], header=None)

channel_df.head()

,subsr,channel,strt_dt,use_tms,asset_nm
0,65460000,MBN,20230908185817,407,MBN 뉴스 7
1,62500000,MBN,20230908231436,12,쇼킹나이트 2부(8회)
2,65183000,MBN,20230908235857,10,쇼킹나이트 2부(8회)
3,66431000,MBN,20230908090506,136,아침 & 매일경제
4,63419000,MBN,20230908225545,1076,쇼킹나이트 1부(8회)


In [ ]:
# contents
# log_dt

content_df = pd.read_excel('/content/drive/MyDrive/hv_project/Data/contents.xlsx')
content_df.drop(['series_nm'], axis=1, inplace=True)
content_df.rename(columns={'super_asset_nm':'asset_nm'}, inplace=True)

content_df.head()

,subsr,asset_nm,ct_cl,genre_of_ct_cl,log_dt
0,66885000,나는자연인이다 516회,TV 시사/교양,인물/다큐,20230917085318
1,66885000,나는자연인이다 516회,TV 시사/교양,인물/다큐,20230920161048
2,64385000,MBC 네트워크 특선 845회,TV 시사/교양,기타,20230922190616
3,64659000,이슈 픽 쌤과 함께 145회,TV 시사/교양,기타,20230908153822
4,64659000,이슈 픽 쌤과 함께 145회,TV 시사/교양,기타,20230908144512


## 데이터 탐색

#### 결측치 확인

In [ ]:
# 데이터 타입
vod_df.isnull().sum()

channel_df.isnull().sum()

# series_nm에 7184 결측치 값이 있음
content_df.isnull().sum()

subsr                0
series_nm         7184
super_asset_nm       0
ct_cl                0
genre_of_ct_cl       0
log_dt               0
dtype: int64

#### 데이터 크기 확인

In [ ]:
# 행 개수
vod_df.shape[0] # 5042
channel_df.shape[0] # 989942
content_df.shape[0] # 13635

#### strt_dt를 datetime으로 바꿔주는 작업

In [ ]:
# strt_dt를 시간으로 변경
vod_df['strt_dt'] = vod_df['strt_dt'].apply(lambda x:pd.to_datetime(str(x),format="%Y%m%d%H%M%S"))

In [ ]:
# 데이터가 너무 많아 시간이 오래 걸림...
# strt_dt를 시간으로 변경
channel_df['strt_dt'] = channel_df['strt_dt'].apply(lambda x:pd.to_datetime(str(x),format="%Y%m%d%H%M%S"))

In [ ]:
# 날짜, 시간, 시, 분 추출
vod_df['date'] = vod_df['strt_dt'].dt.date
vod_df['hour'] = vod_df['strt_dt'].dt.hour
vod_df['min'] = vod_df['strt_dt'].dt.minute

#### 주말, 평일인지 확인하는 코드

In [ ]:
# 평일, 주말 확인

def check_weekend(date):

  # input - 날짜 형식
  day = date.weekday()
  # 날짜 -> 문자열
  date = datetime.strftime(date,"%Y-%m-%d")

  # 리스트 형태로 반환
  kr_holidays = pytimekr.holidays()

  hoilday = False

  for i in kr_holidays:
      if(date == str(i)):
          hoilday = True  # 오늘이 공휴일이면 True

  if day > 4 or hoilday: # 오늘이 주말이거나 공휴일이면
      return 1
  else:
      return 0

In [ ]:
# 데이터를 저장할 list(리스트)를 만듬
weekend = []

# 열에 추가할 각 행을 for로 순회
for i in vod_df['date']:
  result = check_weekend(i)
  if result == 1:
    weekend.append(1)
  else:
    weekend.append(0)

vod_df['is_weekend'] = weekend

In [ ]:
vod_df

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,strt_dt,date,hour,min,is_weekend
0,64659000,과학수사대 스모킹 건 18회(23/08/02),TV 시사/교양,기타,3120,2023-09-02 12:11:24,2023-09-02,12,11,1
1,64536000,(HD)그것이알고싶다 1364회(23/08/12),TV 시사/교양,기타,4260,2023-09-30 10:26:07,2023-09-30,10,26,1
2,66873000,(HD)그것이알고싶다 1366회(23/08/26),TV 시사/교양,기타,4388,2023-09-30 04:21:58,2023-09-30,4,21,1
3,62421000,과학수사대 스모킹 건 13회(23/06/21),TV 시사/교양,기타,3180,2023-09-02 12:34:20,2023-09-02,12,34,1
4,66307000,식객 허영만의 백반기행 206회(23/07/07),TV 시사/교양,인물/다큐,244,2023-09-02 19:26:39,2023-09-02,19,26,1
...,...,...,...,...,...,...,...,...,...,...
5037,66900000,함께 놀자 뽀로로 01회,키즈,기타,720,2023-09-04 12:04:33,2023-09-04,12,4,0
5038,66900000,핑크퐁 뮤지컬 공룡동화 01회,키즈,기타,300,2023-09-04 11:53:27,2023-09-04,11,53,0
5039,66900000,뽀롱뽀롱 뽀로로 NEW 1 02회,키즈,애니메이션,9,2023-09-04 12:49:04,2023-09-04,12,49,0
5040,66900000,뽀롱뽀롱 뽀로로 NEW 1 01회,키즈,애니메이션,420,2023-09-04 12:41:17,2023-09-04,12,41,0


#### subsr마다 취향 확인

In [ ]:
# vod_df의 subsr = 59895000

df2 = vod_df.loc[vod_df['subsr'] == 66900000]
df2

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,strt_dt
252,66900000,(HD)그것이알고싶다 1366회(23/08/26),TV 시사/교양,기타,0,20230920233856
1392,66900000,(HD)궁금한이야기Y 652회(23/09/01),TV 연예/오락,기타,2976,20230922221011
4832,66900000,뽀롱뽀롱 뽀로로 NEW 1 01회,키즈,애니메이션,420,20230909090354
4834,66900000,핑크퐁 자동차 동화 (영어) 01회,키즈,학습,11,20230904120242
4836,66900000,(영어더빙)핑크퐁 이솝이야기 01회,키즈,기타,26,20230904120329
4844,66900000,(영어더빙)뽀롱뽀롱 뽀로로 시즌4 01회,키즈,애니메이션,837,20230904122627
5031,66900000,뽀로로 지구탐험동화 01회,키즈,기타,60,20230904121823
5032,66900000,핑크퐁 뮤지컬 공룡동화 03회,키즈,기타,10,20230904115925
5033,66900000,핑크퐁 뮤지컬 공룡동화 02회,키즈,기타,7,20230904115912
5034,66900000,월간 아기상어(영어) 01회,키즈,학습,46,20230904120057


In [ ]:
df2 = content_df.loc[content_df['subsr'] == 66900000]
df2

,subsr,asset_nm,ct_cl,genre_of_ct_cl,log_dt
12,66900000,그것이알고싶다 1369회,TV 시사/교양,기타,20230920233601
242,66900000,그것이알고싶다 1366회,TV 시사/교양,기타,20230921000739
1226,66900000,궁금한이야기Y 652회,TV 연예/오락,기타,20230922225952
2245,66900000,궁금한이야기Y 654회,TV 연예/오락,기타,20230922220749
8798,66900000,비공식작전,영화,드라마,20230921114330
8868,66900000,비공식작전,영화,드라마,20230907211241
12190,66900000,(예약구매)밀수,영화,액션/어드벤쳐,20230923121447
13632,66900000,핑크퐁 3D 율동 동요 5회,키즈,기타,20230916090106


In [ ]:
# channel_df의 subsr = 59895000

df2 = channel_df.loc[channel_df['subsr'] == 66900000]
df2

NameError: ignored

#### 상위, 하위 카테고리 확인

In [7]:
# 상위 카테고리
# ct_cl 변수 카테고리 확인
ct_cl_cat = vod_df['ct_cl'].astype('category')
# 리스트 변환
ct_list = list(set(ct_cl_cat.values.tolist()))
print(ct_list)

['영화', 'TV 시사/교양', 'TV 연예/오락', 'TV애니메이션', '키즈', 'TV드라마', '다큐', '우리동네']


In [8]:
# 상위 카테고리
# ct_cl 변수 카테고리 확인
content_cl_cat = content_df['ct_cl'].astype('category')
# 리스트 변환
content_list = list(set(content_cl_cat.values.tolist()))
print(content_list)

['영화', '라이프', 'TV 시사/교양', 'TV 연예/오락', '키즈', 'TV애니메이션', 'TV드라마', '스포츠', '다큐', '미분류', '우리동네']


In [9]:
# 하위 카테고리
#genre_of_ct_cl 변수 카테고리 확인
genre_cat = vod_df['genre_of_ct_cl'].astype('category')
# 리스트 변환
genre_list = list(set(genre_cat.values.tolist()))
print(genre_list)

['학원/순정/연애', '명랑/코믹', '추리/미스터리', '코미디', '공포/스릴러', '역사', '오락', '액션/모험', '무협', '액션/어드벤쳐', '로맨틱코미디', 'SF/환타지', '애니메이션', '기타', '인물', '드라마', '멜로', '인물/다큐', '외화 시리즈', '다큐멘터리', '단편', '학습', '연예/오락']


In [10]:
# 상위카테고리별 하위카테고리 분류
vod_dict = {}

for i in ct_list:
  is_ct = vod_df['ct_cl'] == i
  is_ct = vod_df[is_ct]

  genre_cat = is_ct['genre_of_ct_cl'].astype('category')
  genre_cat.values.categories

  vod_dict[i] = list(set(genre_cat.values.tolist()))

print(vod_dict)

{'영화': ['무협', '액션/어드벤쳐', '공포/스릴러', '멜로', '로맨틱코미디', '다큐멘터리', 'SF/환타지', '애니메이션', '역사', '단편', '드라마', '코미디'], 'TV 시사/교양': ['기타', '인물/다큐'], 'TV 연예/오락': ['기타'], 'TV애니메이션': ['학원/순정/연애', '추리/미스터리', '명랑/코믹', '기타', '액션/모험'], '키즈': ['기타', '오락', '학습', '애니메이션'], 'TV드라마': ['기타', '외화 시리즈'], '다큐': ['인물'], '우리동네': ['연예/오락']}


In [11]:
# 상위카테고리별 하위카테고리 분류
content_dict = {}

for i in content_list:
  is_ct = content_df['ct_cl'] == i
  is_ct = content_df[is_ct]

  genre_cat = is_ct['genre_of_ct_cl'].astype('category')
  genre_cat.values.categories

  content_dict[i] = list(set(genre_cat.values.tolist()))

print(content_dict)

{'영화': ['무협', '액션/어드벤쳐', '공포/스릴러', '멜로', '로맨틱코미디', '다큐멘터리', 'SF/환타지', '뮤지컬', '애니메이션', '역사', '단편', '드라마', '서부', '코미디'], '라이프': ['기타'], 'TV 시사/교양': ['기타', '인물/다큐'], 'TV 연예/오락': ['기타'], '키즈': ['기타', '오락', '학습', '애니메이션'], 'TV애니메이션': ['학원/순정/연애', '무협/환타지', '명랑/코믹', '추리/미스터리', '기타', '액션/모험', '스포츠', 'SF/메카'], 'TV드라마': ['기타', '외화 시리즈'], '스포츠': ['기타', '레슬링'], '다큐': ['인물', '자연', '역사'], '미분류': ['미분류'], '우리동네': ['연예/오락']}


In [ ]:
vod_etc_df = vod_df[vod_df['genre_of_ct_cl'] == '기타']
vod_etc_df

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,strt_dt
0,64659000,과학수사대 스모킹 건 18회(23/08/02),TV 시사/교양,기타,3120,20230902121124
1,64536000,(HD)그것이알고싶다 1364회(23/08/12),TV 시사/교양,기타,4260,20230930102607
2,66873000,(HD)그것이알고싶다 1366회(23/08/26),TV 시사/교양,기타,4388,20230930042158
3,62421000,과학수사대 스모킹 건 13회(23/06/21),TV 시사/교양,기타,3180,20230902123420
6,64536000,꼬리에꼬리를무는그날이야기 93회(23/09/07),TV 시사/교양,기타,2601,20230930094215
...,...,...,...,...,...,...
5033,66900000,핑크퐁 뮤지컬 공룡동화 02회,키즈,기타,7,20230904115912
5035,66900000,뽀로로 지구탐험동화 02회,키즈,기타,10,20230904121934
5037,66900000,함께 놀자 뽀로로 01회,키즈,기타,720,20230904120433
5038,66900000,핑크퐁 뮤지컬 공룡동화 01회,키즈,기타,300,20230904115327


## 크롤링

In [ ]:
import re

In [ ]:
vod_df['asset_nm'].unique()

array(['과학수사대 스모킹 건 18회(23/08/02)', '(HD)그것이알고싶다 1364회(23/08/12)',
       '(HD)그것이알고싶다 1366회(23/08/26)', ..., '핑크퐁 뮤지컬 공룡동화 01회',
       '뽀롱뽀롱 뽀로로 NEW 1 02회', '함께 놀자 뽀로로 02회'], dtype=object)

In [ ]:
vod_df['super_asset_nm'] = vod_df['asset_nm'].astype(str).apply(lambda x: x if '회' not in x else re.sub(r'\d+\s*회.*', '', x))

In [ ]:
vod_final = vod_df[['asset_nm','ct_cl','super_asset_nm']]
vod_final

,asset_nm,ct_cl,super_asset_nm
0,과학수사대 스모킹 건 18회(23/08/02),TV 시사/교양,과학수사대 스모킹 건
1,(HD)그것이알고싶다 1364회(23/08/12),TV 시사/교양,(HD)그것이알고싶다
2,(HD)그것이알고싶다 1366회(23/08/26),TV 시사/교양,(HD)그것이알고싶다
3,과학수사대 스모킹 건 13회(23/06/21),TV 시사/교양,과학수사대 스모킹 건
4,식객 허영만의 백반기행 206회(23/07/07),TV 시사/교양,식객 허영만의 백반기행
...,...,...,...
5037,함께 놀자 뽀로로 01회,키즈,함께 놀자 뽀로로
5038,핑크퐁 뮤지컬 공룡동화 01회,키즈,핑크퐁 뮤지컬 공룡동화
5039,뽀롱뽀롱 뽀로로 NEW 1 02회,키즈,뽀롱뽀롱 뽀로로 NEW 1
5040,뽀롱뽀롱 뽀로로 NEW 1 01회,키즈,뽀롱뽀롱 뽀로로 NEW 1


In [ ]:
vod_final['super_asset_nm'].value_counts()

연인 파트1                   149
(HD)런닝맨                  143
최강 배달꾼                   116
연희공략: 건륭황제의여인            110
꼬리에꼬리를무는그날이야기            100
                        ... 
벽 속에 숨은 마법시계(예고)           1
현장법사-서유기의 시작               1
애쉬 래드 2: 황금성을 찾아서(예고)      1
로마: 저항군의 탄생                1
스텝업-브레이킹쓰루                 1
Name: super_asset_nm, Length: 694, dtype: int64

In [ ]:
import re

vod_nm = []

pattern = r'\([^)]*\)'

for i in vod_final['super_asset_nm']:
  i = str(i)
  text = re.sub(pattern=pattern, repl='', string=i)
  vod_nm.append(text)

vod_final['super_asset_nm_final'] = vod_nm

<ipython-input-10-971234d9359c>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vod_final['super_asset_nm_final'] = vod_nm


In [ ]:
import re
li = vod_final['super_asset_nm'].unique()
unique_vod = set()
for i in li:
    if isinstance(i, str):
        matches = re.findall(r'\((.*?)\)', i)
        for match in matches:
            unique_vod.add(f'({match})')
for vod in unique_vod:
    print(vod)

(패키지)
(영어자막)
(9월 이벤트)
(예고)
(영어)
(정규)
(KBS)
(2023)
(상)
(FREE)
(한국판)
(시청각)
(Feat. 원슈타인)
(23/09/08)
(자막)
(노는계집 창)
(1984)
(1977)
(천사데이)
(HD)
(예약구매)
(더빙)
(한국어)
(디지털리마스터링)
(무료)
(6인 6색 캐릭터 영상)
(예고편)
(2022)
(2019)
(2021)
(23/09/19)
(23/07/24)
(SD)
(드라마)
(23/07/28)
(하)
(2020)
(영어더빙)
(부모)
(1985)


In [ ]:
li = vod_final['super_asset_nm'].unique()

vod_unique = []
vod_nm = []


for i in li:
   for vod in unique_vod:
     if vod in i:
        idx.append(i)
        vod_unique.append(vod)
        vod_nm.append(i)

df = pd.DataFrame(zip(vod_unique, vod_nm))
df.columns = ['vod_unique','vod_rm']
df.head()

,vod_unique,vod_rm
0,(HD),(HD)그것이알고싶다
1,(HD),(HD)인간극장
2,(HD),(HD)휴먼다큐 사노라면
3,(HD),(HD)나는자연인이다
4,(HD),(HD)SBS스페셜


In [ ]:
df.to_csv('rename.csv', index=False)

In [ ]:
li = vod_final['super_asset_nm'].unique()

temp = {}

unique_vod = set()

for i in li:
    if isinstance(i, str):
        matches = re.findall(r'\((.*?)\)', i)
        for match in matches:
            unique_vod.add(f'({match})')

for i in li:
  for vod in unique_vod:
    if vod in i:

      temp[vod] = i

df = pd.DataFrame([temp])

df.head()

In [ ]:
vod_final.to_csv('vod_final.csv',index=False)

In [ ]:
vod_title = vod_df[['asset_nm', 'ct_cl']].copy()
vod_title['super_asset_nm'] = vod_df['asset_nm'].str.extract(r'^(.*?)\s\d+회')
vod_title

,asset_nm,ct_cl,super_asset_nm
0,과학수사대 스모킹 건 18회(23/08/02),TV 시사/교양,과학수사대 스모킹 건
1,(HD)그것이알고싶다 1364회(23/08/12),TV 시사/교양,(HD)그것이알고싶다
2,(HD)그것이알고싶다 1366회(23/08/26),TV 시사/교양,(HD)그것이알고싶다
3,과학수사대 스모킹 건 13회(23/06/21),TV 시사/교양,과학수사대 스모킹 건
4,식객 허영만의 백반기행 206회(23/07/07),TV 시사/교양,식객 허영만의 백반기행
...,...,...,...
5037,함께 놀자 뽀로로 01회,키즈,함께 놀자 뽀로로
5038,핑크퐁 뮤지컬 공룡동화 01회,키즈,핑크퐁 뮤지컬 공룡동화
5039,뽀롱뽀롱 뽀로로 NEW 1 02회,키즈,뽀롱뽀롱 뽀로로 NEW 1
5040,뽀롱뽀롱 뽀로로 NEW 1 01회,키즈,뽀롱뽀롱 뽀로로 NEW 1


In [ ]:
vod_title['super_asset_nm'] = vod_title['asset_nm'].str.extract(r'\([^)]*\)')
vod_title

,asset_nm,ct_cl,super_asset_nm
0,과학수사대 스모킹 건 18회(23/08/02),TV 시사/교양,과학수사대 스모킹 건
1,(HD)그것이알고싶다 1364회(23/08/12),TV 시사/교양,(HD)그것이알고싶다
2,(HD)그것이알고싶다 1366회(23/08/26),TV 시사/교양,(HD)그것이알고싶다
3,과학수사대 스모킹 건 13회(23/06/21),TV 시사/교양,과학수사대 스모킹 건
4,식객 허영만의 백반기행 206회(23/07/07),TV 시사/교양,식객 허영만의 백반기행
...,...,...,...
5037,함께 놀자 뽀로로 01회,키즈,함께 놀자 뽀로로
5038,핑크퐁 뮤지컬 공룡동화 01회,키즈,핑크퐁 뮤지컬 공룡동화
5039,뽀롱뽀롱 뽀로로 NEW 1 02회,키즈,뽀롱뽀롱 뽀로로 NEW 1
5040,뽀롱뽀롱 뽀로로 NEW 1 01회,키즈,뽀롱뽀롱 뽀로로 NEW 1


In [ ]:
vod_title['super_asset_nm'] = vod_title['asset_nm'].str.extract(r'^(.*?)\s\d+회')
vod_title

,asset_nm,ct_cl,super_asset_nm
0,과학수사대 스모킹 건 18회(23/08/02),TV 시사/교양,과학수사대 스모킹 건
1,(HD)그것이알고싶다 1364회(23/08/12),TV 시사/교양,(HD)그것이알고싶다
2,(HD)그것이알고싶다 1366회(23/08/26),TV 시사/교양,(HD)그것이알고싶다
3,과학수사대 스모킹 건 13회(23/06/21),TV 시사/교양,과학수사대 스모킹 건
4,식객 허영만의 백반기행 206회(23/07/07),TV 시사/교양,식객 허영만의 백반기행
...,...,...,...
5037,함께 놀자 뽀로로 01회,키즈,함께 놀자 뽀로로
5038,핑크퐁 뮤지컬 공룡동화 01회,키즈,핑크퐁 뮤지컬 공룡동화
5039,뽀롱뽀롱 뽀로로 NEW 1 02회,키즈,뽀롱뽀롱 뽀로로 NEW 1
5040,뽀롱뽀롱 뽀로로 NEW 1 01회,키즈,뽀롱뽀롱 뽀로로 NEW 1


In [ ]:
import re

vod_nm = []

pattern = r'\([^)]*\)'

for i in vod_title['super_asset_nm']:
  i = str(i)
  text = re.sub(pattern=pattern, repl='', string=i)
  vod_nm.append(text)

vod_title['super_asset_nm_final'] = vod_nm



In [ ]:
vod_title

,asset_nm,ct_cl,super_asset_nm,super_asset_nm_final
0,과학수사대 스모킹 건 18회(23/08/02),TV 시사/교양,과학수사대 스모킹 건,과학수사대 스모킹 건
1,(HD)그것이알고싶다 1364회(23/08/12),TV 시사/교양,(HD)그것이알고싶다,그것이알고싶다
2,(HD)그것이알고싶다 1366회(23/08/26),TV 시사/교양,(HD)그것이알고싶다,그것이알고싶다
3,과학수사대 스모킹 건 13회(23/06/21),TV 시사/교양,과학수사대 스모킹 건,과학수사대 스모킹 건
4,식객 허영만의 백반기행 206회(23/07/07),TV 시사/교양,식객 허영만의 백반기행,식객 허영만의 백반기행
...,...,...,...,...
5037,함께 놀자 뽀로로 01회,키즈,함께 놀자 뽀로로,함께 놀자 뽀로로
5038,핑크퐁 뮤지컬 공룡동화 01회,키즈,핑크퐁 뮤지컬 공룡동화,핑크퐁 뮤지컬 공룡동화
5039,뽀롱뽀롱 뽀로로 NEW 1 02회,키즈,뽀롱뽀롱 뽀로로 NEW 1,뽀롱뽀롱 뽀로로 NEW 1
5040,뽀롱뽀롱 뽀로로 NEW 1 01회,키즈,뽀롱뽀롱 뽀로로 NEW 1,뽀롱뽀롱 뽀로로 NEW 1


In [ ]:
vod_title.to_csv('sisa.csv',index=False)